In [1]:
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import shutil

csv_filename = os.path.join(os.getcwd(), 'eval_prediction.csv')
if not os.path.exists(csv_filename):
    df = pd.DataFrame(columns=['img_name', 'img_time', 'gaze_time', 'real_label'])
    df.to_csv(csv_filename, index=False)

img_dir = r'C:\Users\zhangzhida\Desktop\EyeGazePipeline\data\screenshot'
gaze_data_dir = r'C:\Users\zhangzhida\Desktop\EyeGazePipeline\observer'
predict_dir = r'C:\Users\zhangzhida\Desktop\EyeGazePipeline\pipeline'

In [2]:
df_eval = pd.read_csv(csv_filename)
df_eval

,img_name,img_time,gaze_time,real_label


In [3]:
df_gaze_data = pd.read_csv(os.path.join(gaze_data_dir, 'gaze_data.csv'))
df_gaze_data.shape

(2267, 13)

In [4]:
type(df_gaze_data.timestamp[0])

numpy.float64

In [5]:
df_img = pd.read_csv(os.path.join(gaze_data_dir, 'screenshot_list.csv')) 
df_img.shape

(43, 2)

In [6]:
def eval_res(df_img, df_gaze_data):   
    delay_threshold = 0.01
    i = 0
    j = 0 
    cnt = 0
    while i < df_img.shape[0] and j < df_gaze_data.shape[0]:
        img = df_img.iloc[i]
        gaze_record = df_gaze_data.iloc[j]
        
        gaze_point_time = gaze_record['timestamp']
        img_time = img['time']
        if gaze_point_time != gaze_point_time:
            j += 1
            continue
            
        diff_time = gaze_point_time - img_time
        # print(i, j, diff_time)
        if abs(diff_time) < delay_threshold:
            print(i, j, diff_time)

            # decide if left or right
            # if gaze_record['FPOGY'] is None: continue
            if gaze_record['FPOGX'] < 0.5:
                df_eval.loc[cnt] = [img['img_name'], img_time, gaze_point_time, 'left']
            else:
                df_eval.loc[cnt] = [img['img_name'], img_time, gaze_point_time, 'right']
            cnt += 1
            i += 1

        if gaze_point_time < img_time:
            j += 1
        else:
            i += 1
        
    return df_eval

In [7]:
df_eval =  eval_res(df_img, df_gaze_data)

5 174 0.006981611251831055
7 180 0.0
9 193 0.0
11 445 -0.006981849670410156
12 458 0.007978677749633789
15 654 0.0009992122650146484
17 666 -0.0069811344146728516
18 892 -0.003988504409790039
23 1125 0.0
28 1535 -0.0019953250885009766
30 1542 0.006974220275878906
32 1554 -0.0019948482513427734
34 1789 0.003987789154052734
36 1801 -0.0019941329956054688
37 1804 -0.005985736846923828
38 2030 0.0
40 2042 -0.006979703903198242
41 2043 0.005983114242553711


In [10]:
df_predict = pd.read_csv(os.path.join(predict_dir, 'prediction.csv')) 
#df_predict['img_name'] = df_predict['img_name'].map(lambda x: str(x)[:-1])
df_predict

,img_name,predict_label
0,1573559789.3222148.jpeg,1
1,1573559789.820947.jpeg,1
2,1573559789.923674.jpeg,1
3,1573559790.0273943.jpeg,1
4,1573559790.13012.jpeg,1
5,1573559794.3235643.jpeg,1
6,1573559794.4252927.jpeg,1
7,1573559794.5330067.jpeg,1
8,1573559794.6377256.jpeg,1
9,1573559794.7424455.jpeg,1


In [11]:
res = pd.merge(df_eval, df_predict, on='img_name')
res

,img_name,img_time,gaze_time,real_label,predict_label
0,1573559794.3235643.jpeg,1.573560e+09,1.573560e+09,left,1
1,1573559794.5330067.jpeg,1.573560e+09,1.573560e+09,left,1
2,1573559794.7424455.jpeg,1.573560e+09,1.573560e+09,left,1
3,1573559799.426047.jpeg,1.573560e+09,1.573560e+09,left,1
4,1573559799.6364849.jpeg,1.573560e+09,1.573560e+09,left,1
5,1573559804.5285287.jpeg,1.573560e+09,1.573560e+09,left,1
6,1573559804.738965.jpeg,1.573560e+09,1.573560e+09,left,1
7,1573559809.5284457.jpeg,1.573560e+09,1.573560e+09,right,1
8,1573559814.7367077.jpeg,1.573560e+09,1.573560e+09,left,1
9,1573559824.3292816.jpeg,1.573560e+09,1.573560e+09,left,1
